In [1]:
import pandas as pd

data = pd.read_csv('data/train.csv')

data = data.drop(["Name", "Ticket","Cabin"], axis="columns")

data = data.dropna(axis = "index", how = "any")
data = data.reset_index(drop = True)

In [3]:
data

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...,...
707,886,0,3,female,39.0,0,5,29.1250,Q
708,887,0,2,male,27.0,0,0,13.0000,S
709,888,1,1,female,19.0,0,0,30.0000,S
710,890,1,1,male,26.0,0,0,30.0000,C


In [8]:
from sklearn import preprocessing

In [9]:
def transform_dummy(data):
    enc = preprocessing.OneHotEncoder()
    enc.fit(data)
    encdata = enc.transform(data).toarray()
    encdata = pd.DataFrame(encdata)
    columns=[]
    for i in range(data.shape[1]):
        columns = columns + enc.categories_[i].tolist()
    encdata.columns = columns
    encdata.index = data.index
    return encdata

In [10]:
transform_dummy(data.loc[:,("Sex","Embarked")])

,female,male,C,Q,S
0,0.0,1.0,0.0,0.0,1.0
1,1.0,0.0,1.0,0.0,0.0
2,1.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,1.0
4,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...
707,1.0,0.0,0.0,1.0,0.0
708,0.0,1.0,0.0,0.0,1.0
709,1.0,0.0,0.0,0.0,1.0
710,0.0,1.0,1.0,0.0,0.0


In [15]:
data = pd.concat([data.drop(["Sex", "Embarked"], axis = "columns"), transform_dummy(data.loc[:,("Sex","Embarked")])], axis = "columns")

In [16]:
data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare',
       'female', 'male', 'C', 'Q', 'S'],
      dtype='object')

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
X ,y= data.drop(["PassengerId", "Survived"], axis = "columns"), data["Survived"]

In [19]:
train_X, test_X, train_y, test_y =train_test_split(X, y, test_size=0.3)

In [20]:
from sklearn.ensemble import RandomForestClassifier

In [21]:
from sklearn.model_selection import GridSearchCV

In [22]:
import numpy as np

In [28]:
params = {"n_estimators":[*range(1,11)]
         ,"criterion":("gini","entropy")
         ,"max_depth":[*range(1,11)]
         ,"min_samples_leaf":[*range(1,51,5)]
         ,"min_impurity_decrease":[*np.linspace(0,1,11)]
         }

In [29]:
clf = RandomForestClassifier(random_state=0,verbose=1,bootstrap=False)

In [30]:
gs = GridSearchCV(clf, params,n_jobs=-1,cv = 3, verbose=1
                 ,return_train_score=True)

In [31]:
gs = gs.fit(X, y)

Fitting 3 folds for each of 22000 candidates, totalling 66000 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s finished


In [32]:
print(gs.best_params_)#返回最佳组合
print(gs.best_score_)

{'criterion': 'entropy', 'max_depth': 10, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 6, 'n_estimators': 4}
0.8188904253684597


In [41]:
from sklearn.model_selection import cross_val_score

In [91]:
clf = RandomForestClassifier(random_state=0,verbose=1
                            ,criterion="entropy"
                            ,max_depth=10
                            ,min_impurity_decrease=0
                            ,min_samples_leaf=6
                            ,n_estimators=10000
                            ,n_jobs=-1)

clf = clf.fit(train_X, train_y)

clf.score(test_X,test_y)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 3192 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 4042 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done 4992 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 6042 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done 7192 tasks      | elapsed:    8.4s
[Parallel(n_jobs=-1)]: Done 8442 tasks      | elapsed:    9.9s
[Parallel(n_jobs=-1)]: Done 9792 tasks      | elapsed:   11.8s
[Parallel(n_jobs=-1)]: Done 10000 out of 

0.8084112149532711

In [62]:
data_test = pd.read_csv("data/test.csv")

In [63]:
data_test = data_test.drop(["Name", "Ticket","Cabin"], axis="columns")

In [69]:
data_test.isnull().sum()

PassengerId     0
Pclass          0
Age            86
SibSp           0
Parch           0
Fare            1
female          0
male            0
C               0
Q               0
S               0
dtype: int64

In [67]:
data_test = pd.concat([data_test.drop(["Sex","Embarked"],axis = "columns")
                       ,transform_dummy(data_test.loc[:,["Sex","Embarked"]])], axis = "columns")

In [83]:
data_test.head()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,female,male,C,Q,S
0,892.0,3.0,34.5,0.0,0.0,7.8292,0.0,1.0,0.0,1.0,0.0
1,893.0,3.0,47.0,1.0,0.0,7.0000,1.0,0.0,0.0,0.0,1.0
2,894.0,2.0,62.0,0.0,0.0,9.6875,0.0,1.0,0.0,1.0,0.0
3,895.0,3.0,27.0,0.0,0.0,8.6625,0.0,1.0,0.0,0.0,1.0
4,896.0,3.0,22.0,1.0,1.0,12.2875,1.0,0.0,0.0,0.0,1.0


In [84]:
X.head()

,Pclass,Age,SibSp,Parch,Fare,female,male,C,Q,S
0,3,22.0,1,0,7.2500,0.0,1.0,0.0,0.0,1.0
1,1,38.0,1,0,71.2833,1.0,0.0,1.0,0.0,0.0
2,3,26.0,0,0,7.9250,1.0,0.0,0.0,0.0,1.0
3,1,35.0,1,0,53.1000,1.0,0.0,0.0,0.0,1.0
4,3,35.0,0,0,8.0500,0.0,1.0,0.0,0.0,1.0


In [75]:
from sklearn.impute import SimpleImputer
 
imp = SimpleImputer(missing_values=np.nan, strategy='mean')

In [77]:
imp.fit(data_test)

SimpleImputer()

In [80]:
data_test = pd.DataFrame(imp.transform(data_test), columns= data_test.columns)

In [92]:
pd.DataFrame({"PassengerId":data_test.loc[:,"PassengerId"],
              "Survived":clf.predict(data_test.drop("PassengerId",axis = "columns")).tolist()}).to_csv("submission.csv")

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 1242 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 1792 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 2442 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Done 3192 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done 4042 tasks      | elapsed:    0.9s
[Parallel(n_jobs=4)]: Done 4992 tasks      | elapsed:    1.1s
[Parallel(n_jobs=4)]: Done 6042 tasks      | elapsed:    1.3s
[Parallel(n_jobs=4)]: Done 7192 tasks      | elapsed:    1.6s
[Parallel(n_jobs=4)]: Done 8442 tasks      | elapsed:    1.8s
[Parallel(n_jobs=4)]: Done 9792 tasks      | elapsed:    2.1s
[Parallel(n_jobs=4)]: Done 10000 out of 10000 | elapsed: